# Test the folium application
Make foliumm maps that include the EOdal generated S2 images

In [1]:
# load packages
import folium
import numpy as np
import leafmap
import rasterio as rio
import branca.colormap as cm
import ipywidgets as widgets
from matplotlib import colors as colors
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import date
from glob import glob

from ipywidgets import interact, fixed
from IPython.display import display

### list available files

In [2]:
# User inputs
aoi_name = "SH"
satellite_tiffs_dir = Path(f"./{aoi_name}")

path_str = satellite_tiffs_dir.absolute()
filelist = [x for x in path_str.glob("*.tiff")]
date_list = np.sort([x.stem.split('_')[0] for x in filelist])

### Widgets

In [3]:
dropdown_band_selection = widgets.Dropdown(
    options=['RGB', 'NDVI'],
    value='RGB',
    description='Display:',
    disabled=False,
)

date_slider = widgets.SelectionSlider(
    options=date_list,
    value=date_list[-1],
    description='Select a date ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

In [4]:
%matplotlib inline
@interact(selected_date=date_slider, band_selection=dropdown_band_selection)
def show_s2_image(selected_date: date, band_selection: str):

    fname = f"{selected_date}_{aoi_name}_WGS84.tiff"
    path_infile_tiff = satellite_tiffs_dir.joinpath(fname)

    # read in the created TIFF to allow geolocalization
    with rio.open(path_infile_tiff) as src:
        boundary = src.bounds
        img = src.read()  
        img[img == 1e+20] = 0

    # calculate the center of the supplied TIFF
    calc_lat = (boundary.bottom + boundary.top)/2
    calc_lon = (boundary.left + boundary.right)/2

    # calculate the boundaries used by ImageOverlay (SW, NE)
    img_bounds = ((boundary.bottom, boundary.left), (boundary.top, boundary.right))

    # Folium expects (Y, X, BANDS) format
    if band_selection == 'RGB':
        # rearrange the image
        img_rearranged = np.moveaxis(img[0:3,:,:], 0, -1)
        # get the mask
        mask = img_rearranged[:,:,0] == 0

        temp_img = np.ones(shape=[img_rearranged.shape[0], img_rearranged.shape[1], 4], dtype=np.uint8)
        temp_img[:, :, 1:4] = (img_rearranged*255).astype(np.uint8)
        temp_img[mask, 0] = 255
        
    elif band_selection == 'NDVI':
        temp_img = (img[-1,:,:]*255).astype(np.uint8)

    img_for_plot = temp_img
    center = (calc_lat, calc_lon)

    m = folium.Map(center, zoom_level=15)

    if band_selection == 'RGB':
        folium.raster_layers.ImageOverlay(
            image=img_for_plot[:, :, 1::],
            name=fname,
            bounds=img_bounds,
            opacity=1.
        ).add_to(m)

    elif band_selection == 'NDVI':
        folium.raster_layers.ImageOverlay(
            image=img_for_plot,
            name=fname,
            bounds=img_bounds,
            opacity=1.
        ).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)

interactive(children=(SelectionSlider(continuous_update=False, description='Select a date ', index=3, options=…